# 导入包和数据

In [1]:
import pandas as pd
import numpy
import gurobipy as gp
import json
import timeit

# read data
with open("sorted_ToyData_dict.json") as json_file:
     toy_data_dict= json.load(json_file)
with open("location.json") as json_file:
     location = json.load(json_file)
with open("slot.json") as json_file:
     slot = json.load(json_file)
with open("data_req.json") as json_file:
     demand = json.load(json_file)
with open("bandwidth.json") as json_file:
     bandwidth = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: '../small_data/6/sorted_ToyData_dict.json'

# 初始化常量

In [2]:
# number of jobs and slots
w = len(toy_data_dict)
J = len(slot)

# number of tasks in a job in stage 1
L = []
for key in toy_data_dict.keys():
    L.append(len(toy_data_dict[key]['stage']['1']))

# slots number in a data center
a = []
for val in slot.values():
    a.append(val)

# all tasks in stage 1
tasks_1 = []
for i in toy_data_dict.keys():
    tasks_1.append(toy_data_dict[i]['stage']['1'])
    
# all tasks
tasks = []
for i in toy_data_dict.keys():
    _task = []
    for j in toy_data_dict[i]['Execution Time'].keys():
        _task.append(j)
    tasks.append(_task)
    
# normalize location
for (key,val) in location.items():
    s = val[2:]
    location[key] = int(s)

# execution time of certain tasks
num = 0
e = []
for i in toy_data_dict.keys():
    exc = []
    for j in toy_data_dict[i]['Execution Time'].keys():
        if j in tasks_1[num]:
            exc.append(toy_data_dict[i]['Execution Time'][j])
    e.append(exc)
    num += 1
    
# communication time matrix
c = []
for k in range(w):
    com = []
    for i in range(len(tasks_1[k])):
        compute = []
        for j in range(J):
            maximum = 0
            for loc in demand[tasks_1[k][i]]:
                src = location[loc] - 1
                time = demand[tasks_1[k][i]][loc]/bandwidth[src][j]
                if(time > maximum):
                    maximum = time
            compute.append(maximum)
        com.append(compute)
    c.append(com)
    
# completion time records
task_complete_time = {}
complete_time = {}
job_complete_time = {}

# the bottleneck task
bottle_task = {}

# completed job records
complete_jobs = []

# record global time
global_time = 0

# record current stages and total stages
stages = [1] * w
total_stages = []
s = ''
for i in toy_data_dict.keys():
    for j in toy_data_dict[i]['stage'].keys():
        s = int(j)
    total_stages.append(s)
    
# slots occupied by jobs
occ_slots = {}

# record the slot available time
slots_empty = {}
for j in range(J):
    slots_empty[j] = []

In [3]:
# set thresholds for rescheduling
time_threshold = 1
job_threshold = 1

# 线性求解接口实现

In [4]:
def LP(K, L, avai, M):
    # define model
    m = gp.Model()
    
    # add variables
    x = [0] * w
    X = [0] * w
    for k in K:
        x[k] = m.addVars(L[k],J,2,name="lambda"+str(k))
        X[k] = m.addVars(L[k],J,name="x"+str(k))
        
    # update model
    m.update()
    
    # set objective
    m.setObjective(
    gp.quicksum(x[k][i,j,0]+x[k][i,j,1]*pow(M,c[k][i][j]+e[k][i])
            for k in K for i in range(L[k]) for j in range(J)),
    gp.GRB.MINIMIZE
    )
    
    # add constraints
    m.addConstrs(X[k][i,j] == x[k][i,j,1] 
            for k in K for i in range(L[k]) for j in range(J))
    m.addConstrs(x[k][i,j,0] + x[k][i,j,1] == 1
            for k in K for i in range(L[k]) for j in range(J))
    m.addConstrs(gp.quicksum(X[k][i,j] for k in K for i in range(L[k]))
            <= avai[j] for j in range(J))
    m.addConstrs(gp.quicksum(X[k][i,j] for j in range(J)) == 1
            for k in K for i in range(L[k]))
    
    # solve it
    m.optimize()
    
    # res records the assigned tasks locations
    res = []
    for var in m.getVars():
        if(var.X == 1.0 and var.varName[0] == 'x'):
            res.append(int(var.varName[5:-1])+1)
    return res

In [5]:
def update(K, avai, M):
    num = 0
    maxim = 0
    bottleneck = 0
    bottle = 0
    
    # find the task with the longest time
    for i in K:
        for j in range(len(tasks_new[i])):
            time = c[i][j][res[num]-1] + e[i][j]
            if(time > maxim):
                maxim = time
                bottleneck = i
                bottle = j
            num += 1
    complete_time[bottleneck] = global_time + maxim
    bottle_task[bottleneck] = bottle
    
    # find all task in the longest job
    occ_slots[bottleneck] = []
    setted = []
    num = 0
    for i in K:
        for j in range(len(tasks_new[i])):
            if(i == bottleneck):
                task_complete_time[tasks_new[i][j]] = c[i][j][res[num]-1] + e[i][j] + global_time
                setted.append(res[num]-1)
                occ_slots[bottleneck].append(res[num]-1)
                slots_empty[res[num] - 1].append(c[i][j][res[num]-1] + e[i][j] + global_time)
                location[tasks_new[i][j]] = res[num]
            num += 1
    
    # remove the longest job from the scheduling queue
    K.remove(bottleneck)
    M = 0
    
    # update M and available slots
    for i in K:
        M += J * L[i]
    for i in setted:
        avai[i] = avai[i] - 1
    return K, avai, M

# 串行剪枝实现

In [6]:
# to get the time of a empty slot
def findTask(loc):
    time = 1000
    task = ""
    job = 0
    for i in slots_empty[loc]:
        if(i < time):
            time = i
    for (key, val) in task_complete_time.items():
        if(val == time):
            task = key
    for i in tasks:
        if task in i:
            break
        job += 1
    return time, job

In [7]:
# update the job complete time
def updateCompleteTime(k):
    complete_time[k] = 0
    for i in tasks_new[k]:
        if(task_complete_time[i] > complete_time[k]):
            complete_time[k] = task_complete_time[i]
    return complete_time

In [8]:
# to find a better serial solution
def prune():
    complete_time_new = complete_time.copy()

    for k in bottle_task.keys():
        i = bottle_task[k]
        t = tasks_new[k][i]
        loc = location[t] - 1
        # find if there is a better solution for the longest task
        for j in range(J):
            if(c[k][i][j] < c[k][i][loc]):
                time, task = findTask(j)
                if(time + c[k][i][j] < c[k][i][loc] + global_time or avai[j] > 0):
                    # assign the task to the new slot
                    print("find a better location!")
                    print(t)
                    print(location[t])
                    print(j+1)
                    print("Original time:"+str(c[k][i][loc] + global_time))
                    print("Current time:"+str(time+c[k][i][j]))
                    print(slots_empty)
                    
                    slots_empty[j].remove(time)
                    slots_empty[loc].remove(task_complete_time[t])
                    location[t] = j+1
                    avai[loc] += 1
                    occ_slots[k].remove(loc)
                    occ_slots[k].append(j)
                    occ_slots[task].remove(j)
                    
                    task_complete_time[t] = time + c[k][i][j] + e[k][i]
                    slots_empty[j].append(time + c[k][i][j] + e[k][i])
                    complete_time_new = updateCompleteTime(k)
    return location, task_complete_time, complete_time_new, occ_slots, avai, slots_empty

# 多轮调度接口实现

In [9]:
def release_resource():
    # sort the jobs by their complete time
    complete_time_order = sorted(complete_time.items(), key=lambda x:x[1])
    
    # define help variables
    bottle_task.clear()
    num = 0
    wait = 0
    start = complete_time_order[0][1]
    remained_jobs = w - len(job_complete_time)
    
    # update the completed jobs according to time and job thresholds
    complete_jobs.clear()
    for i in range(min(job_threshold,remained_jobs)):
        wait = (complete_time_order[i][1] - start)
        if(wait > time_threshold):
            global_time = start + time_threshold
            break
        global_time = complete_time_order[i][1]
        complete_jobs.append(complete_time_order[i][0])
        
    # find completed jobs and release the slots that have been occupied by the completed tasks
    num = 0
    remove = []
    for i in complete_jobs:
        
        if(stages[i] == total_stages[i]):
            remove.append(i)
            job_complete_time[i] = complete_time_order[num][1]
            del complete_time[i]
        
        for j in occ_slots[i]:
            avai[j] += 1
        
        cnt = 0
        for key in toy_data_dict.keys():
            if(cnt == i):
                complete_tasks = toy_data_dict[key]['stage'][str(stages[i])]
                break
            cnt += 1
        
        for j in complete_tasks:
            slots_empty[location[j]-1].remove(task_complete_time[j])
        occ_slots[i].clear()
        num += 1
        stages[i] += 1
    
    for i in remove:
        complete_jobs.remove(i)
        
    return global_time, complete_jobs, stages, job_complete_time, avai, occ_slots, slots_empty

In [10]:
def set_next_iteration():
    # initialize variables
    tasks_new.clear()
    K = complete_jobs.copy()
    num = 0
    
    # add new stages of completed jobs to the scheduling pool
    for i in toy_data_dict.keys():
        L[num] = 0
        if num in complete_jobs:
            tasks_new.append(toy_data_dict[i]['stage'][str(stages[num])])
            L[num] = len(toy_data_dict[i]['stage'][str(stages[num])])
        else:
            tasks_new.append([])
        num += 1
        
    # reset M for new tasks to be scheduled
    M = 0
    for i in K:
        M += J * L[i]
    
    # reset e for new tasks 
    e.clear()
    num = 0
    for i in toy_data_dict.keys():
        exc = []
        for j in toy_data_dict[i]['Execution Time'].keys():
            if j in tasks_new[num]:
                exc.append(toy_data_dict[i]['Execution Time'][j])
        e.append(exc)
        num += 1
        
    # reset c for new tasks
    c.clear()
    for k in range(w):
        com = []
        for i in range(len(tasks_new[k])):
            compute = []
            for j in range(J):
                maximum = 0
                for loc in demand[tasks_new[k][i]]:
                    src = location[loc] - 1
                    time = demand[tasks_new[k][i]][loc]/bandwidth[src][j]
                    if(time > maximum):
                        maximum = time
                compute.append(maximum)
            com.append(compute)
        c.append(com)
        
    return K, tasks_new, L, M, e, c

# 全过程调度

In [11]:
# 第一轮数据初始化
K = []
for i in range(w):
    K.append(i)

avai = a.copy()

M = 0
for i in K:
    M += J*L[i]
    
tasks_new = tasks_1.copy()

In [12]:
# 第一轮迭代
for i in range(w):
    res = LP(K,L,avai,M)
    K, avai, M = update(K, avai, M)
location, task_complete_time, complete_time_new, occ_slots, avai, slots_empty = prune()

Academic license - for non-commercial use only - expires 2021-07-25
Using license file /Users/emiyali/gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 229 rows, 312 columns and 624 nonzeros
Model fingerprint: 0xfdfcecbd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+14]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 208 rows and 208 columns
Presolve time: 0.01s
Presolved: 21 rows, 104 columns, 208 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0400000e+02   8.000000e+00   0.000000e+00      0s
       8    1.7322726e+09   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds
Optimal objective  1.732272583e+09
Gurobi Optimizer ve

In [13]:
# 全过程
while(len(complete_time) != 0):
    global_time, complete_jobs, stages, job_complete_time, avai, occ_slots, slots_empty = release_resource()
    K, tasks_new, L, M, e, c = set_next_iteration()
    for i in range(len(complete_jobs)):
        res = LP(K,L,avai,M)
        K, avai, M = update(K, avai, M)
    location, task_complete_time, complete_time_new, occ_slots, avai, slots_empty = prune()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 67 rows, 78 columns and 156 nonzeros
Model fingerprint: 0x34f94cfd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 67 rows and 78 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1334208e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  5.133420803e+03
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 39 columns and 78 nonzeros
Model fingerprint: 0xa009265e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+04]
  Bou

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.8421358e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.842135784e+06


# 结果一览

In [14]:
job_complete_time

{0: 2.0,
 1: 5.511111111111111,
 2: 7.576388888888889,
 3: 7.861111111111111,
 4: 13.38888888888889,
 5: 22.624603174603177}

In [47]:
task_complete_time

{'tE1': 4.315789473684211,
 'tF1': 4.25,
 'tC1': 4.131578947368421,
 'tB1': 3.0,
 'tD1': 2.7894736842105265,
 'tD2': 2.769230769230769,
 'tA1': 1.9411764705882353,
 'tA2': 1.9411764705882353,
 'tD3': 4.733918128654971,
 'tD4': 4.092503987240829,
 'tB2': 4.833333333333334,
 'tC2': 5.231578947368421,
 'tF2': 6.458333333333334,
 'tF3': 5.65,
 'tF4': 7.4939024390243905,
 'tE2': 6.580495356037152,
 'tE3': 7.473684210526316,
 'tD5': 6.246738641475484,
 'tC3': 7.310526315789474,
 'tE4': 9.81410974244121,
 'tE5': 8.623684210526315,
 'tF5': 8.788020086083215,
 'tF6': 9.164634146341463,
 'tF7': 12.099416755037115,
 'tF8': 11.581300813008129,
 'tE6': 13.077267637178052,
 'tF9': 16.170845326465688}

In [ ]:
location